In [ ]:
import numpy
import pandas
from src.data_objects import AllData
from src.functions import *
from src.readers import ReadDesign

retrain = True
runchain = True
logTrain = True

ThisData = AllData
ThisData["name"] = "epem"

model_par = "input/epemDesign.txt"
outdir = "/data/rjfgroup/rjf01/cameron.parker/tunes/" + ThisData["name"] + "/"

Trimming points and obs ranges

In [ ]:
del ThisData["Observables"]["PrPr2760"]
del ThisData["Observables"]["PrPr13000"]

ThisData["Design"] = ReadDesign("/data/rjfgroup/rjf01/cameron.parker/runs/LEPdesign/QVir_Analysis/parameters.txt")

trimRanges(ThisData)

In [ ]:
badpoints = []
trimPoints(badpoints,ThisData)

Making Data pkl for selected observables

In [ ]:
buildDataPkl(ThisData, logTrain)
print(ThisData["datapkl"])

Getting emulators

In [ ]:
from src.emulator_BAND import EmulatorBAND

setEmuPaths(ThisData)

if retrain:
    buildObsPkls(ThisData)
    trainEmulators(model_par, ThisData, logTrain)
else:
    readEmulators(ThisData)

Running Chain

In [ ]:
from src.mcmc import Chain
import os

mcmcpath = "mcmc/" + ThisData["name"] + "-chain.pkl"
mymcmc = Chain(mcmc_path=mcmcpath, expdata_path=ThisData["datapkl"], model_parafile=model_par)
mymcmc.loadEmulator(getEmuPathList(ThisData))

In [ ]:
os.environ["OMP_NUM_THREADS"] = "20"
# may have to: export RDMAV_FORK_SAFE=1 before running the code

n_effective=4000
n_active=2000
n_prior=8000
sample="tpcn"
n_max_steps=100
random_state=42

n_total = 25000
n_evidence = 0

pool = 20

if runchain:
    sampler = mymcmc.run_pocoMC(n_effective=n_effective, n_active=n_active,
                            n_prior=n_prior, sample=sample,
                            n_max_steps=n_max_steps, random_state=random_state,
                            n_total=n_total, n_evidence=n_evidence, pool=pool)

Corner Plot

In [ ]:
import pickle
import corner
import matplotlib.pyplot as plt
import numpy as np
        
with open(mcmcpath, 'rb') as pf:
        data = pickle.load(pf)

labels = mymcmc.label
print(labels)

fig = corner.corner(data['chain'], weights=data['weights'], labels=labels, color="C0")
plt.show()

In [ ]:
TransformedSamples = np.copy(data['chain'])
TransformedSamples[:,0] = data['chain'][:,0]
TransformedSamples[:,1] = (2*data['chain'][:,3]+0.05) + (data['chain'][:,0]-(2*data['chain'][:,3]+0.05))*data['chain'][:,1]
TransformedSamples[:,2] = data['chain'][:,2]
TransformedSamples[:,3] = data['chain'][:,3]
TransformedSamples[:,4] = data['chain'][:,4]
TransformedSamples[:,5] = data['chain'][:,5]

labels[1] = "QS"
fig = corner.corner(TransformedSamples, weights=data['weights'], labels=labels, color="C0")
plt.show()
fig.savefig(outdir+'Corner.pdf', dpi = 192)

bestParams = ee_extract_parameters(mymcmc, ThisData["Design"]["Parameter"], outdir)

Priors

In [ ]:
makeplot(ThisData, "Priors", outdir, logTrain=logTrain)

Posteriors

In [ ]:
makeplot(ThisData, "Posteriors", outdir, samples=data["chain"], logTrain=logTrain)

Validation

In [ ]:
from src.data_objects import valData
valData["Design"] = ReadDesign("/data/rjfgroup/rjf01/cameron.parker/runs/validation/LEP/QVir_Analysis/parameters.txt")
trimRanges(valData)
validationPlots(valData, ThisData, outdir, logTrain=logTrain)